In [ ]:
# ECB CODE THINGS
from hashlib import md5
from base64 import *
from Crypto.Cipher import AES

block_size = 16
pad = lambda a: a+(block_size - len(a)% block_size)*\
    chr(block_size-len(a)% block_size)
unpad = lambda a: a[:-ord(a[len(a)-1:])]

class AESCipher:

    def __init__(self,key):
        self.key = md5(key.encode('utf8')).hexdigest()

    def encrypt(self, raw):
        raw = pad(raw)
        cipher = AES.new(self.key.encode("utf8"), AES.MODE_ECB)
        return b64encode(cipher.encrypt(raw.encode('utf8')))

    def decrypt(self, enc):
        enc = b64decode(enc)
        cipher = AES.new(self.key.encode("utf8"), AES.MODE_ECB)
        return unpad(cipher.decrypt(enc)).decode('utf8')



    


In [ ]:
##
# MAIN
# Just a test.
msg = input('Hello World')
pwd = input('1234')

print('Ciphertext:', AESCipher(pwd).encrypt(msg))
b = ('Ciphertext:', AESCipher(pwd).encrypt(msg))
# print('Ciphertext:', AESCipher(pwd).decrypt(b))

In [ ]:
# CBC CODE THINGS
from hashlib import md5
from base64 import b64decode
from base64 import b64encode
import os
import struct

from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
from Crypto.Util.Padding import pad, unpad

#useless comment

# class AESCipher:
#     def __init__(self, key):
#         self.key = md5(key.encode('utf8')).digest()

#     def encrypt(self, data):
#         iv = get_random_bytes(AES.block_size)
#         self.cipher = AES.new(self.key, AES.MODE_CBC, iv)
#         return b64encode(iv + self.cipher.encrypt(pad(data.encode('utf-8'), 
#             AES.block_size)))

#     def decrypt(self, data):
#         raw = b64decode(data)
#         self.cipher = AES.new(self.key, AES.MODE_CBC, raw[:AES.block_size])
#         return unpad(self.cipher.decrypt(raw[AES.block_size:]), AES.block_size)


# if __name__ == '__main__':
#     print('TESTING ENCRYPTION')
#     print(get_random_bytes(16))
#     msg = input('Message...: ')
#     pwd = input('Password..: ')
#     print('Ciphertext:', AESCipher(pwd).encrypt(msg).decode('utf-8'))

#     print('\nTESTING DECRYPTION')
#     cte = input('Ciphertext: ')
#     pwd = input('Password..: ')
#     print('Message...:', AESCipher(pwd).decrypt(cte).decode('utf-8'))


key = get_random_bytes(16)
iv = get_random_bytes(16)
cbc = AES.new(key,AES.MODE_CBC,iv)
fsz=os.path.getsize('./mustang.bmp')
with open('ciphertext.txt','wb') as fout:
    fout.write(struct.pack('<Q',fsz))
#fout.write(iv)
sz=2048

with open('../Projects/mustang.bmp') as fin:
    while True:
        data = fin.read(sz)
        n=len(data)
        if n==0:break
        elif n%16 ==0:
            data += ' ' * (16 - n % 16)
            encd = cbc.encrypt(data)
            fout.write(encd)





In [3]:
import Crypto
import numpy as np
from Crypto.Cipher import AES

def main():
    key = key_gen()
    iv = IV_gen()
    header_size = 54
    info = [key, iv]

#Comment out either of the two sets of code below to test the cbc and ecb 
    # encrypt_file('mustang.bmp','out1.bmp', header_size, 'cbc',info)
    # decrypt_file('out1.bmp','out2.bmp', header_size, 'cbc',info)

    # encrypt_file('mustang.bmp','out1.bmp', header_size, 'ecb',info)
    # decrypt_file('out1.bmp','out2.bmp', header_size, 'ecb',info)

def aes_ecb_encrypt(key, plaintext,block_size):
    max_val = len(plaintext) - 1
    cipher = AES.new(key,AES.MODE_ECB)
    pos = 0
    result = b''

    while((pos + block_size-1) <= max_val):
        result += get_ciphertext(cipher, plaintext[pos:pos+block_size])
        pos+=block_size
    
    tmp = plaintext[pos:] + b'\0'*(block_size-len(plaintext[pos:]))
    result += get_ciphertext(cipher,tmp)
    return result


def aes_ecb_decrypt(key,ciphertext,block_size):
    max_val = len(ciphertext) - 1
    cipher = AES.new(key,AES.MODE_ECB)
    pos = 0
    result = b''

    while((pos + block_size-1) <= max_val):
        result += get_plaintext(cipher, ciphertext[pos:pos+block_size])
        pos+=block_size
    
    # tmp = ciphertext[pos:] + b'\0'*(block_size-len(ciphertext[pos:]))
    tmp = ciphertext[pos:]
    result += get_plaintext(cipher,tmp)
    return result

def aes_cbc_encrypt(key, plaintext, block_size, IV):
    max_val = len(plaintext) - 1
    cipher = AES.new(key,AES.MODE_ECB)
    pos = 0
    result = b''
    prev_cipher = IV

    while((pos + block_size-1) <= max_val):
        tmp = byte_xor(plaintext[pos:pos+block_size], prev_cipher)
        prev_cipher = get_ciphertext(cipher, tmp)
        result += prev_cipher
        pos+=block_size
    
    tmp = byte_xor(prev_cipher, (plaintext[pos:] + b'\0'*(block_size-len(plaintext[pos:]))))
    result += get_ciphertext(cipher,tmp)
    return result

def aes_cbc_decrypt(key, ciphertext, block_size, IV):
    max_val = len(ciphertext) - 1
    cipher = AES.new(key,AES.MODE_ECB)
    pos = 0
    result = b''
    prev_cipher = IV

    while((pos + block_size-1) <= max_val):
        tmp = get_plaintext(cipher, ciphertext[pos:pos+block_size])
        plaintext = byte_xor(tmp, prev_cipher)

        result += plaintext
        prev_cipher = ciphertext[pos:pos+block_size]
        pos+=block_size
    
    tmp = get_ciphertext(cipher,ciphertext[pos:])
    plaintext = byte_xor(prev_cipher, tmp)

    result += plaintext
    return result

def get_ciphertext(cipher,plaintext):
    cipher_text = cipher.encrypt(plaintext)
    return cipher_text

def get_plaintext(cipher,ciphertext):
    plaintext = cipher.decrypt(ciphertext)
    return plaintext

def byte_xor(b1,b2):
    return bytes([a ^ b for a, b in zip(b1,b2)])

def open_file(fileName, header_size):
    with open(fileName,'rb') as f:
        bytes = f.read()
    f.close()

    header = bytes[:header_size]
    data = bytes[header_size:]

    return header,data

def key_gen():
    return Crypto.Random.get_random_bytes(16)

def IV_gen():
    return Crypto.Random.get_random_bytes(16)

def encrypt_file(inputFilename, outputFilename, header_size, mode, info):
    header,data = open_file(inputFilename, header_size)
    key = info[0]

    if mode.lower() == 'cbc':
        encrypted = aes_cbc_encrypt(key, data, 16, info[1])
    else: 
        encrypted = aes_ecb_encrypt(key, data, 16)
    
    enc_bytes = header + encrypted
    with open(outputFilename,'wb') as f1:
        f1.write(enc_bytes)
    f1.close()

    print(info)

def decrypt_file(inputFilename, outputFilename, header_size, mode, info):
    header,data = open_file(inputFilename, header_size)
    key = info[0]

    if mode.lower() == 'cbc':
        decrypted = aes_cbc_decrypt(key, data, 16, info[1])
    else:    
        decrypted = aes_ecb_decrypt(key, data, 16)

    dec_bytes = header + decrypted
    with open(outputFilename,'wb') as f2:
        f2.write(dec_bytes)
    f2.close()

    print(info)

if __name__ == '__main__':
    main()




[b'oap2Uzyc\xaa=\xbeGan\xc3\xf6', b'2\xc4\xd4\x9d\x11*\xd8\xf5\xe3\xa0\xf2\x88\xf1H\xff\x15']
[b'oap2Uzyc\xaa=\xbeGan\xc3\xf6', b'2\xc4\xd4\x9d\x11*\xd8\xf5\xe3\xa0\xf2\x88\xf1H\xff\x15']
